In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from numpy import array
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import pickle
from keras.preprocessing.sequence import pad_sequences
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense, RepeatVector, TimeDistributed
from keras.models import Model, Sequential
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
# print(os.listdir("../input"))
from keras.callbacks import ModelCheckpoint

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
<ipython-input-1-61290da95df1>:29: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
lines=pd.read_csv("Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [3]:
lines['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [4]:
lines=lines[lines['source']=='ted']

In [5]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,"to turn on the lights or to bring him a glass of water,","लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [6]:
pd.isnull(lines).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [7]:
lines=lines[~pd.isnull(lines['english_sentence'])]
lines

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
...,...,...,...
127595,ted,is if we want that to become our reality -,अगर हम चाहते हैं कि यह वास्तविकता बने -
127597,ted,Africa has not done bad.,अफ़्रीका ने कुछ गलती नहीं की है।
127598,ted,Thank you.,धन्यवाद |
127603,ted,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।


In [8]:
lines.drop_duplicates(inplace=True)

In [9]:
lines

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
...,...,...,...
127592,ted,Because when you are a developing country,क्योंकि जब आप एक विकासशील देश हैं
127595,ted,is if we want that to become our reality -,अगर हम चाहते हैं कि यह वास्तविकता बने -
127597,ted,Africa has not done bad.,अफ़्रीका ने कुछ गलती नहीं की है।
127603,ted,and put it in our cheeks.,और अपने गालों में डाल लेते हैं।


In [10]:
lines=lines.sample(n=10240,random_state=42)
lines.shape

(10240, 3)

In [11]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [12]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.replace('”', ''))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.replace('”', ''))

In [13]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.replace('“', ''))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.replace('“', ''))

In [14]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.replace('♫', ''))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.replace('♫', ''))

In [15]:

lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.replace('।', ''))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.replace('।', ''))

In [16]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.replace('™', ''))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.replace('™', ''))

In [17]:
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [18]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [19]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [20]:
# # Add start and end tokens to target sequences
# lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [21]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines["english_sentence"]:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines["hindi_sentence"]:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [22]:
len(lines["english_sentence"])

10240

In [23]:
len(lines["hindi_sentence"])

10240

In [24]:
lines

,source,english_sentence,hindi_sentence
82040,ted,we still dont know who her parents are who she is,हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है
85038,ted,no keyboard,कोई कुंजीपटल नहीं
58018,ted,but as far as being a performer,लेकिन एक कलाकार होने के साथ
74470,ted,and this particular balloon,और यह खास गुब्बारा
122330,ted,and its not as hard as you think integrate climate solutions into all of your innovations,और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें
...,...,...,...
89388,ted,and it also happens to us,और यह हमारे साथ भी होता है
70104,ted,before i started my day challenges,मेरी इस दिन की चुनौतियों को प्रारंभ करने से पहले
89058,ted,a pure and innocent nature a boy who saw the world,बहुत ही शुद्ध और सरल विचार एक लड़का जो जमाने को
80942,ted,in which she would live,जहाँ वो अब रहेगी


In [25]:
hin_excluded_words = sorted(all_hindi_words)[::-1][:1]

In [26]:
for word in sorted(all_hindi_words)[:153]:
    hin_excluded_words.append(word)

In [27]:
hin_excluded_words

['\u200b\u200bकि',
 'a',
 'ab',
 'accelerometer',
 'ackermann',
 'aids',
 'alain',
 'alliance',
 'ap',
 'arthur',
 'as',
 'attention',
 'b',
 'blackberry',
 'bollingbroke',
 'botton',
 'c',
 'cdc',
 'character',
 'cindy',
 'clarke',
 'co',
 'd',
 'daniel',
 'darpa',
 'darwin',
 'date',
 'de',
 'deficit',
 'diego',
 'disorder',
 'doing',
 'dynamic',
 'east',
 'eastland',
 'engine',
 'experimental',
 'ferry',
 'firewire',
 'foundation',
 'francais',
 'function',
 'g',
 'gallop',
 'gartner',
 'gcse',
 'gehrig',
 'gg',
 'ghemawat',
 'giussani',
 'global',
 'globalizers',
 'goldfarb',
 'google',
 'gotjunk',
 'grassroot',
 'gyros',
 'hips',
 'hiv',
 'hydras',
 'icyball',
 'ii',
 'imf',
 'imfअंतर्राष्ट्रिय',
 'inkjet',
 'inside',
 'insula',
 'iphone',
 'iq',
 'island',
 'iv',
 'jinke',
 'joan',
 'kallikuppam',
 'kbs',
 'kinematics',
 'km',
 'laboratory',
 'led',
 'lewin',
 'lhc',
 'long',
 'lou',
 'machine',
 'mapmaker',
 'mars',
 'masiosare',
 'mit',
 'mr',
 'mri',
 'new',
 'nfl',
 'now',
 '

In [28]:
eng_excluded_words = sorted(all_eng_words)[::-1][:46]

In [29]:
hin_excluded_words = set(hin_excluded_words)
eng_excluded_words = set(eng_excluded_words)

In [30]:
eng_indices = set()

for eng in lines["english_sentence"]:
    for word in eng.split():
        if word in eng_excluded_words:
            eng_indices.add(lines["english_sentence"][lines["english_sentence"] == eng].index[0])

# len(eng_indices)

In [31]:
hin_indices = set()
for hin in lines["hindi_sentence"]:
    for word in hin.split():
        if word in hin_excluded_words:
            hin_indices.add(lines["hindi_sentence"][lines["hindi_sentence"] == hin].index[0])

# hin_indices

In [32]:
indices_to_be_deleted = eng_indices.union(hin_indices)

In [33]:
len(indices_to_be_deleted)

163

In [34]:
lines.drop(labels="source", inplace=True, axis=1)

In [35]:
for index in indices_to_be_deleted:
    lines["english_sentence"].drop(inplace=True, index=index, axis=0)
    lines["hindi_sentence"].drop(inplace=True, index=index, axis=0)

In [36]:
print(len(lines["english_sentence"]), len(lines["hindi_sentence"]))

10077 10077


In [37]:
lines["hindi_sentence"].shape

(10077,)

In [38]:
lines = pd.concat([lines["english_sentence"], lines["hindi_sentence"]], axis=1)

In [39]:
lines.to_excel(excel_writer="./nmt_processes.xlsx")

In [40]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [41]:
print(len(lines["english_sentence"]), len(lines["hindi_sentence"]))

10077 10077


In [42]:
sorted(lines['hindi_sentence'])

['START_ अंग्रेज़ीबोलने वाले हर तरह के कार्टूनिस्ट _END',
 'START_ अंग्रेजी भाषा और वर्णमाला का सभी ज्ञान _END',
 'START_ अंग्रेजी भाषा साम्राज्यवाद की भाषा के रूप में देखी गयी थी _END',
 'START_ अंग्रेजी में इसके अलावा कि डीएनए अणु की अनुचित प्रतिकृति से _END',
 'START_ अंग्रेजी में विशाल सुधार देखे _END',
 'START_ अंडे और हम्मस पीसे छोले के लिए होता है _END',
 'START_ अंत में आप देखें यह कहाँ जा रहा है _END',
 'START_ अंत में बेंजामिन और जुलियन ने फैसला किया कि _END',
 'START_ अंत में वो कहता है देखो क्या तुम अपने घर के सब खिड़की दरवाज़े खोल कर _END',
 'START_ अंतरिक्ष के कुछ भाग हम अपनी आकाशगंगा के कारण नहीं देख पाते _END',
 'START_ अंतरिक्ष में जीवन खोजने के लिए _END',
 'START_ अंतरितक्षयान की निकट भविष्य में वापसी की प्रतीक्षा करनी होगी _END',
 'START_ अंतर्राष्ट्रिय समुदाय से _END',
 'START_ अंदाजन सिर्फ़ अफ़्रीका में ही _END',
 'START_ अंधी अपढ़ आज भी हूँ _END',
 'START_ अकड से चलता हुआ नेताओं की तरह मोबाइल लहराता हुआ नही होगा _END',
 'START_ अक्तूबर में विभिन्न देशों से से अधिक

In [43]:
lines.shape

(10077, 2)

In [44]:
### Get English and Hindi Vocabulary
all_eng_words_new=set()
for eng in lines["english_sentence"]:
    for word in eng.split():
        if word not in all_eng_words_new:
            all_eng_words_new.add(word)

all_hindi_words_new=set()
for hin in lines["hindi_sentence"]:
    for word in hin.split():
        if word not in all_hindi_words_new:
            all_hindi_words_new.add(word)

In [45]:
input_words = sorted(list(all_eng_words_new))
target_words = sorted(list(all_hindi_words_new))

In [46]:
sorted(all_eng_words_new)[::-1]

['zooming',
 'zoom',
 'zero',
 'zen',
 'zeitgeist',
 'zealands',
 'zamin',
 'z',
 'yves',
 'yuyu',
 'yugoslav',
 'yr',
 'youve',
 'youtube',
 'youth',
 'yourselves',
 'yourself',
 'yours',
 'youre',
 'your',
 'younger',
 'young',
 'youll',
 'youd',
 'you',
 'yosemite',
 'yoruba',
 'york',
 'yoricks',
 'yoohoo',
 'ylang',
 'yield',
 'yet',
 'yesterday',
 'yes',
 'yep',
 'yellow',
 'years',
 'yearolds',
 'yearold',
 'yearning',
 'year',
 'yeah',
 'yaws',
 'yards',
 'yams',
 'yak',
 'yachts',
 'xinhua',
 'xerox',
 'x',
 'wrote',
 'wrongness',
 'wrong',
 'written',
 'writing',
 'writes',
 'writers',
 'writer',
 'write',
 'wristwatches',
 'wretched',
 'wreck',
 'wrapped',
 'wrap',
 'wowwww',
 'wow',
 'wouldnt',
 'would',
 'worthy',
 'worthless',
 'worthiness',
 'worth',
 'worst',
 'worship',
 'worsen',
 'worse',
 'worrying',
 'worry',
 'worried',
 'worn',
 'worms',
 'worldwide',
 'worlds',
 'worldquality',
 'worldclass',
 'world',
 'workstyle',
 'workshops',
 'workshop',
 'works',
 'workpla

In [47]:
sorted(all_hindi_words_new)[::-1]

['ह्रास',
 'ह्रदय',
 'ह्म्म',
 'हौसलाअफ़ज़ाई',
 'हौडिंग',
 'हौकमोथएक',
 'हौकमोथ',
 'होर्न',
 'होमोसेपिएन्स',
 'होम',
 'होनेके',
 'होने',
 'होनी',
 'होना',
 'होनहार',
 'होद',
 'होते',
 'होती',
 'होता',
 'होटल',
 'होगे',
 'होगी',
 'होगा',
 'होग',
 'होकर',
 'होऊंगा',
 'होउंगी',
 'होंडुरास',
 'होंगेपत्रकार',
 'होंगे',
 'होंगी',
 'होंग',
 'हों',
 'हो',
 'हॊगा',
 'हॉल',
 'हॉटडॉग्स',
 'हैहम',
 'हैस्वस्थ',
 'हैसिर्फ',
 'हैश',
 'हैरिस',
 'हैरानी',
 'हैरान',
 'हैराजनीति',
 'हैरत',
 'हैये',
 'हैयहाँ',
 'हैमबर्गर',
 'हैम',
 'हैफिर',
 'हैदुर्घटना',
 'हैदराबाद',
 'हैतुम्हे',
 'हैती',
 'हैट',
 'हैजा',
 'हैअपने',
 'हैंहँसी',
 'हैंडबैग',
 'हैंउसका',
 'हैं',
 'हैँ',
 'है',
 'हेसा',
 'हेलीकाप्टर',
 'हेलियम',
 'हेलिकोप्टर',
 'हेर',
 'हेमोर्हेज',
 'हेप',
 'हेनसन',
 'हेतु',
 'हेती',
 'हेतान',
 'हेड्गर',
 'हेडर',
 'हेड',
 'हेज़',
 'हेक्सागन',
 'हेक्स',
 'हेंगरों',
 'हेंगर',
 'हें',
 'हे',
 'हृदयसेहृदय',
 'हृदयरोग',
 'हृदय',
 'हूबहू',
 'हूप',
 'हूंहूंहूंहूं',
 'हूं',
 'हूँसहानुभूति',
 'हूँमेरा',
 'हूँ',
 'हू'

In [48]:
lines['length_eng_sentence'] = lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence'] = lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [49]:
lines[lines['length_eng_sentence']>30].shape

(0, 4)

In [50]:
lines[lines['length_eng_sentence']<=30].shape

(10077, 4)

In [51]:
lines=lines[lines['length_eng_sentence']<=30]
lines=lines[lines['length_hin_sentence']<=30]

In [52]:
lines["length_hin_sentence"].shape

(10077,)

In [53]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  30
maximum length of English Sentence  20


In [54]:
max_length_tar=max(lines['length_hin_sentence'])
max_length_src=max(lines['length_eng_sentence'])

In [55]:
max_length_tar

30

In [56]:
max_length_src

20

In [57]:
with open("nmt_data.pickle", "wb") as nmt_data:
    pickle.dump(lines, nmt_data)

In [58]:
with open("variable.pickle", "wb") as variable:
    pickle.dump([all_hindi_words_new, all_eng_words_new, max_length_tar, int(max_length_src)], variable)

In [59]:
sorted(all_hindi_words_new)

['START_',
 '_END',
 'अ',
 'अँगरेज़',
 'अँग्रेज़ी',
 'अँग्रेजी',
 'अँग्रेजों',
 'अँधकारयुग',
 'अँधेरा',
 'अँधेरे',
 'अंक',
 'अंकगणित',
 'अंकल',
 'अंकित',
 'अंकीय',
 'अंकुरित',
 'अंग',
 'अंगूठा',
 'अंगूठी',
 'अंगूर',
 'अंग्रेज़',
 'अंग्रेज़ी',
 'अंग्रेज़ीबोलने',
 'अंग्रेजी',
 'अंजान',
 'अंजीर',
 'अंटार्कटिका',
 'अंडे',
 'अंत',
 'अंतःकरण',
 'अंतड़ियों',
 'अंतत',
 'अंततः',
 'अंतर',
 'अंतरराष्ट्रीय',
 'अंतरराष्ट्रीयकरण',
 'अंतराल',
 'अंतरिक्ष',
 'अंतरितक्षयान',
 'अंतर्गत',
 'अंतर्ग्रहीय',
 'अंतर्राष्ट्रिय',
 'अंतवस्त्र',
 'अंतहीन',
 'अंतिम',
 'अंदर',
 'अंदाजन',
 'अंदाज़',
 'अंदाज़ा',
 'अंधकार',
 'अंधकारयुक्त',
 'अंधविश्वास',
 'अंधी',
 'अंधेपन',
 'अंधेरा',
 'अंधेरे',
 'अंबार',
 'अंश',
 'अकड',
 'अकल्पनीय',
 'अक़्सर',
 'अकाउंट',
 'अकादमी',
 'अकाल',
 'अकेला',
 'अकेली',
 'अकेले',
 'अक्टूबर',
 'अक्तूबर',
 'अक्यूमन',
 'अक्रा',
 'अक्ल',
 'अक्ष',
 'अक्षर',
 'अक्षरसः',
 'अक्षरों',
 'अक्सर',
 'अखबार',
 'अखबारों',
 'अख़बार',
 'अख़बारों',
 'अखाडा',
 'अगर',
 'अगरबत्तियां',
 'अगला',
 'अगली',
 'अगले',
 'अ